modbus to JSON conversion

# data

In [9]:
data_1 = [0, 3, 0, 0, 0, 0, 0, 0, 32768, 0, 0, 0, 0, 1, 34, 16, 41, 0, 5, 24, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 147, 161, 165, 176, 175, 174, 175, 31, 34, 36, 43, 43, 34, 0, 0, 16880, 0, 16231, 7282, 17008, 0, 15921, 50973, 17946, 3174, 17933, 59833, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 16601, 39322, 16512, 0, 32704, 0, 32704, 0, 0, 0, 0, 0, 0, 0, 16055, 21846, 17740, 52019, 0, 0, 17065, 0, 15860, 29127, 17209, 32768, 17191, 41444]
data_2 = [0, 3, 0, 0, 0, 0, 0, 0, 32768, 0, 0, 0, 0, 1, 34, 16, 41, 0, 30, 24, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 81, 80, 81, 82, 112, 152, 149, 31, 34, 36, 43, 43, 34, 0, 0, 0, 0, 16231, 7282, 0, 0, 15921, 50973, 17946, 3174, 17933, 59833, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 32704, 0, 32704, 0, 32704, 0, 32704, 0, 0, 0, 16057, 12136, 17740, 52019, 0, 0, 0, 0, 15853, 2426, 17133, 39322, 17170, 58766]
data_3 = [0, 3, 0, 0, 0, 0, 0, 0, 32768, 0, 0, 0, 0, 1, 34, 16, 41, 0, 35, 24, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 80, 81, 80, 80, 107, 146, 142, 31, 34, 37, 43, 42, 34, 0, 0, 0, 0, 16226, 19418, 0, 0, 15918, 4855, 17946, 3174, 17933, 59833, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 32704, 0, 32704, 0, 32704, 0, 32704, 0, 0, 0, 16038, 43690, 17740, 52019, 0, 0, 0, 0, 15860, 29127, 17120, 52429, 17169, 16450]
len(data_1)

104

# data extraction

In [274]:
import csv
import math
import struct
import json
from collections import OrderedDict

In [280]:
def adjust_offset():
    """
    Read and adjust offset value
    Known bug: children offset < parent offset will not work
    
    Send:
        float: next offset
    Yields:
        float: absolute offset relative to position 0
    """
    cur = parent = prev = offset = 0
    while True:
        inc = cur - prev
        if inc < 0:
            parent = prev
        #print('g cur: {}, inc: {}, par: {}'.format(cur, inc, parent))
        prev = cur
        if cur < parent:
            cur = yield parent + cur, parent
        else:
            cur = yield cur, None
            parent = cur

def unpack_bool(data):
    """Extract list of boolean from an integer
    
    Args:
        data (int)
    Return:
        list: in bool
    """
    return [bool(int(i)) for i in '{:016b}'.format(data)]

def decimals_to_float32(int1, int2):
    """Convert to float
    
    Note:
        Orders in big endian
    Args:
        int1, int2 (int): two interger to combine
    Return:
        float: converted number
    """
    f = int('{:016b}{:016b}'.format(int1, int2), 2)
    return struct.unpack('f', struct.pack('I', f))[0]

def unpack_real(data, offset):
    """Get data and convert to float
    
    Args:
        data (list): List of data
        offset (int): starting address
    Return:
        number in float
    """
    return decimals_to_float32(data[offset//2], data[offset//2+1])

def get_val(data, position, data_type):
    """
    Args:
        data (list): data list
        position (float): position to get
        data_type (string): type of data
    Return:
        coorsponding value at the position
    """
    if data_type == 'Int':
        return data[int(position)//2]
    
    elif data_type == 'Bool':
        bool_pos = round(10*(position-math.floor(position)))
        if math.floor(position)%2 == 1: # add position for boolean at offset in odd positions
            bool_pos += 8
        return unpack_bool(data[math.floor(position//2)])[bool_pos]
    
    elif data_type == 'Real':
        return unpack_real(data, int(position))
    
    elif data_type == 'Struct' or 'Array' in data_type:
        return {}
    
    

"""adjust_offset test and example:"""
rel_offsets = [0, 2, 4, 6, 0.0, 0.1, 0.2, 0.3, 0.4, 1.1, 8, 10, 0,   4,  8]
abs_offsets = [0, 2, 4, 6, 6.0, 6.1, 6.2, 6.3, 6.4, 7.1, 8, 10, 10, 14, 18]
ao_test = adjust_offset()
next(ao_test)
for i in range(len(rel_offsets)):
    assert ao_test.send(float(rel_offsets[i]))[0] == abs_offsets[i], 'adjust offset'

"""unpack_bool test and example:"""
assert unpack_bool(32768)[0], 'True at first val'
assert any(unpack_bool(32768)[1:]) == False, 'False at the rest'


In [268]:
# milestone 1: basic converting
with open('15103EZM_DB201_1.csv', 'r') as f:
    reader = csv.reader(f, doublequote=True, quoting=csv.QUOTE_ALL, escapechar='\\')
    
    print_format = '{:12.12} {:9.9}|abs_pos: {:4.4}|par: {}|ex Value: {}'
    ao = adjust_offset()
    next(ao)
    rel_offset = abs_offset = par_offset = 0
    
    for row in reader:
        '''0: name, 1: type, 2: offset'''
        rel_offset = row[2]
        abs_offset, par_offset = ao.send(float(rel_offset))
        ex_value = get_val(data_1, abs_offset, row[1])

        try:
            value = data_1[int(rel_offset)//2]
        except:
            #print('error')
            value = 'Error'
        print(print_format.format(row[0], '({})'.format(row[1]), rel_offset, parent, ex_value))

Job_Num      (Int)    |abs_pos: 0   |par: 192.0|ex Value: 0
MC_st        (Int)    |abs_pos: 2   |par: 192.0|ex Value: 3
EX1_status   (Int)    |abs_pos: 4   |par: 192.0|ex Value: 0
EX2_status   (Int)    |abs_pos: 6   |par: 192.0|ex Value: 0
LSP_in_prese (Int)    |abs_pos: 8   |par: 192.0|ex Value: 0
Spark_H_Acc  (Int)    |abs_pos: 10  |par: 192.0|ex Value: 0
Spark_C_Acc  (Int)    |abs_pos: 12  |par: 192.0|ex Value: 0
pb           (Struct) |abs_pos: 14  |par: 192.0|ex Value: -other-
start        (Bool)   |abs_pos: 0   |par: 192.0|ex Value: False
stop         (Bool)   |abs_pos: 0.1 |par: 192.0|ex Value: False
inc_ex1      (Bool)   |abs_pos: 0.2 |par: 192.0|ex Value: False
dec_ex1      (Bool)   |abs_pos: 0.3 |par: 192.0|ex Value: False
inc_ex2      (Bool)   |abs_pos: 0.4 |par: 192.0|ex Value: False
dec_ex2      (Bool)   |abs_pos: 0.5 |par: 192.0|ex Value: False
inc_lsp      (Bool)   |abs_pos: 0.6 |par: 192.0|ex Value: False
dec_lsp      (Bool)   |abs_pos: 0.7 |par: 192.0|ex Value: False
st

In [284]:
# milestone 2: creating a dictionary
data = data_2 

data_extracted = OrderedDict()

with open('15103EZM_DB201_1.csv', 'r') as f:
    reader = csv.reader(f, doublequote=True, quoting=csv.QUOTE_ALL, escapechar='\\')
    
    ao = adjust_offset()
    next(ao)
    
    rel_offset = abs_offset = par_offset = 0
    parent_name = ""
    
    for row in reader:
        '''row: [name, type, offset]'''
        rel_offset = row[2]
        abs_offset, par_offset = ao.send(float(rel_offset))
        ex_value = get_val(data, abs_offset, row[1])
        
        if par_offset == None:
            parent_name = row[0]
            data_extracted[row[0]] = ex_value            
        else:
            data_extracted[parent_name][row[0]] = ex_value

print(json.dumps(data_extracted, indent=4))

{
    "Job_Num": 0,
    "MC_st": 3,
    "EX1_status": 0,
    "EX2_status": 0,
    "LSP_in_present": 0,
    "Spark_H_Acc": 0,
    "Spark_C_Acc": 0,
    "pb": {
        "start": false,
        "stop": false,
        "inc_ex1": false,
        "dec_ex1": false,
        "inc_ex2": false,
        "dec_ex2": false,
        "inc_lsp": false,
        "dec_lsp": false
    },
    "st_ind": {
        "EX1_ok": true,
        "EX1_war": false,
        "EX1_err": false,
        "EX2_ok": false,
        "EX2_war": false,
        "EX2_err": false,
        "MC_ok": false,
        "MC_war": false,
        "MC_err": false
    },
    "ex1_SPD_in_lamp": {
        "10%": false,
        "20%": false,
        "30%": false,
        "40%": false,
        "50%": false,
        "60%": false,
        "70%": false,
        "80%": false,
        "90%": false,
        "100%": false
    },
    "ex2_SPD_in_lamp": {
        "10%": false,
        "20%": false,
        "30%": false,
        "40%": false,
        "50%": fal

# helper function test area

In [10]:
import struct

def decimals_to_float32(int1, int2):
    '''Convert to float
    
    Note:
        Orders in big endian
    Args:
        int1, int2: two interger to combine
    Return:
        number in float
    '''
    f = int('{:016b}{:016b}'.format(int1, int2), 2)
    return struct.unpack('f', struct.pack('I', f))[0]

def unpack_real(data, offset):
    '''Get data and convert to float
    
    Args:
        data: List of data
        offset: starting address
    Return:
        number in float
    '''
    return decimals_to_float32(data[offset//2], data[offset//2+1])

print(unpack_real(data_1, 96))
print(unpack_real(data_1, 104))
print(unpack_real(data_1, 108))
print(unpack_real(data_1, 200))
print(unpack_real(data_1, 204))

0.9027777910232544
0.173611119389534
9859.099609375
185.5
167.63238525390625


In [58]:

def unpack_bool(data):
    '''Get the boolean data
    
    Args:
        data: integer
        position: location of bool to extract
    Return:
        list of bool
    '''
    return [bool(int(i)) for i in '{:016b}'.format(data)]

for i in range(16):
    print(unpack_bool(32768)[i])
    


True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False


# scrape area

In [37]:
#st = 'abc\x69\xe6\x03'
#bytes(st, 'latin1') # Note that 'ascii' encoding gives error for some values.
st.encode('latin1')

b'abci\xe6\x03'

[9984, 37908, 20, 12288, 0, 0, 0, 36890, 0, 49271, 49152, 64, 0, 4, 128, 0, 1400, 18, 1520, 65534, 1400, 1, 1400, 63, 1050, 2, 1400, 5, 15000, 25, 155, 0, 0, 0, 46, 28, 0, 0, 1, 0, 60, 0, 35, 0, 0, 0, 0, 0, 1, 0, 1491, 1577, 1622, 1757, 1750, 1748, 238, 245, 262, 324, 341, 327, 1804, 1615, 32767, 0, 1804, 16, 0, 0, 0, 17287, 0, 17056, 0, 17056, 0, 17234, 0, 16928, 0, 16928, 0, 16959, 39322, 17053, 0, 0, 0, 17008, 0, 16226, 19418, 17829, 34816, 0, 0, 18706, 31744, 17585, 30588, 17941, 0]

In [2]:
'{:016b}'.format(9984)

'0010011100000000'

In [6]:
print(int('0010011100000000', 2))

9984


In [21]:
data_new = [9984, 37908, 20, 4096, 0, 0, 0, 36890, 0, 50700, 49152, 64, 0, 4, 0, 0, 1400, 18, 1520, 65535, 1400, 1, 1400, 67, 1050, 2, 1400, 5, 15000, 27, 155, 0, 0, 0, 46, 28, 0, 0, 1, 0, 61, 0, 34, 0, 0, 0, 0, 0, 1, 0, 1494, 1604, 1662, 1734, 1747, 1749, 240, 247, 263, 326, 338, 319, 1772, 1607, 32767, 0, 1772, 16, 0, 0, 0, 17287, 0, 17056, 0, 17056, 0, 17234, 0, 16928, 0, 16928, 0, 16672, 0, 17056, 0, 0, 0, 17008, 0, 16235, 60681, 0, 0, 0, 0, 18706, 31744, 17585, 30588, 17941, 0]

print(len(data_new))
for i in range(17,21):
    print(data_new[i])
    
print(decimals_to_float32(data_new[99], data_new[100]))
print(decimals_to_float32(data_new[101], data_new[102]))



103
18
1520
65535
1400
1419.73388671875
9536.0
